In [207]:
import spacy
import tensorflow as tf
from tqdm import tqdm
import tensorflow_text as text
import tensorflow_hub as hub
import pandas as pd

In [112]:
train_df = pd.read_csv('./input/train.csv')

In [4]:
nlp = spacy.load('en_core_web_lg')

In [7]:
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [50]:
DISCOURSE_TYPE = tf.constant(['Lead' ,'Position' , 'Claim' , 'Counterclaim' ,'Rebuttal' ,'Evidence' ,'Concluding Statement'])
DISCOURSE_TYPE_LABELS = tf.constant([1,2,3,4,5,6,7])

TARGET = tf.constant(['Adequate', 'Effective','Ineffective'])

TARGET_LABELS = tf.constant([0,1,2])

def get_label(Keys,values,features):
    init = tf.lookup.KeyValueTensorInitializer(Keys,values)
    table = tf.lookup.StaticHashTable(init,default_value=-1)
    return table.lookup(tf.constant(features))
# layer = tf.keras.layers.StringLookup(vocabulary=TARGET,output_mode="one_hot",num_oov_indices=0)
# table.lookup(tf.constant(['Adequate', 'Effective','Ineffective'])).numpy()
 

In [43]:
y_train = tf.one_hot(get_label( TARGET, TARGET_LABELS, train_df['discourse_effectiveness']),3)

In [103]:
y_train

<tf.Tensor: shape=(36765, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)>

In [51]:
y_discourse_type = get_label( DISCOURSE_TYPE, DISCOURSE_TYPE_LABELS, train_df['discourse_type'])

In [66]:
train_df['discourse_type_label'] = y_discourse_type

In [67]:
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,discourse_type_label
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,1
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,2
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,3
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,6
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,4


In [1]:
def get_word2vec(input):
    input = [nlp(str(x)).vector for x in tqdm(input)]
    return input
     

In [ ]:
train_df['Adequate'] = pd.get_dummies(train_df['discourse_effectiveness'])['Adequate']
train_df['Effective'] = pd.get_dummies(train_df['discourse_effectiveness'])['Effective']
train_df['Ineffective'] = pd.get_dummies(train_df['discourse_effectiveness'])['Ineffective']

In [ ]:
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Adequate,Effective,Ineffective
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,1,0,0
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,1,0,0
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,1,0,0
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,1,0,0
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,1,0,0


In [ ]:
train_df.drop(['discourse_id','essay_id','discourse_type','discourse_effectiveness'],axis=1,inplace=True)

In [ ]:
train_df.head()

,discourse_text,Adequate,Effective,Ineffective
0,"Hi, i'm Isaac, i'm going to be writing about h...",1,0,0
1,"On my perspective, I think that the face is a ...",1,0,0
2,I think that the face is a natural landform be...,1,0,0
3,"If life was on Mars, we would know by now. The...",1,0,0
4,People thought that the face was formed by ali...,1,0,0


In [ ]:
from sklearn.model_selection import train_test_split

X = train_df['discourse_text']
y = train_df[['Adequate','Effective','Ineffective']]

X_train,X_valid, y_train, y_vaild = train_test_split(X,y,test_size=0.2)

In [ ]:
X_train = tf.data.Dataset.from_tensors(X_train)
X_valid = tf.data.Dataset.from_tensors(X_valid)
y_train = tf.data.Dataset.from_tensors(y_train)
y_vaild = tf.data.Dataset.from_tensors(y_vaild)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36765 entries, 0 to 36764
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   discourse_text  36765 non-null  object
 1   Adequate        36765 non-null  uint8 
 2   Effective       36765 non-null  uint8 
 3   Ineffective     36765 non-null  uint8 
dtypes: object(1), uint8(3)
memory usage: 395.1+ KB


In [ ]:
# tf_dfas = tf.data.Dataset.from_tensor_slices()
layer1 = Lambda(lambda x : get_word2vec(x),name='word2vec')

layer2 = Bidirectional(GRU(512,return_sequences=True,input_shape=[None,1]))


In [ ]:
layer2(trasad)

ValueError: Input 0 of layer "bidirectional_11" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (5, 300)

In [ ]:
from keras import backend as K

trasad = layer1(X_train[:5])

100%|██████████| 5/5 [00:00<00:00, 96.56it/s]


In [ ]:
travas = layer1(X_valid[:5])

100%|██████████| 5/5 [00:00<00:00, 46.43it/s]


In [ ]:
layer2(trasad)

ValueError: Unexpectedly found an instance of type `<class 'numpy.ndarray'>`. Expected a symbolic tensor instance.

In [ ]:
for x in X_train:
    print(x)

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import LSTM,GRU,TimeDistributed,Dense,Lambda,Input,concatenate,Embedding,Flatten,Bidirectional

vocab_size = 5000
num_oov = 1000
embed_size = 100

model = tf.keras.Sequential([
    # Lambda(get_word2vec,name='word2vec'),
    # Embedding(vocab_size+num_oov,embed_size),
    Bidirectional(GRU(512,return_sequences=True,input_shape=[None,1])),
    GRU(256,input_shape=[None,1]),
    Dense(3,activation="sigmoid")
])



# input_A = Input(shape=[1], name="discourse_type_label")
# input_B = Input(shape=[None], name = "discourse_text")

# text_vector = Lambda(get_word2vec,name='word2vec')(input_B)
# embed_layer = Embedding(vocab_size+num_oov,embed_size,input_shape=[None])(input_B)
# hidden_GRU_1 = GRU(512,return_sequences=True)(embed_layer)
# hidden_GRU_2 = GRU(256,return_sequences=True)(hidden_GRU_1)
# hidden_1 = Flatten()(hidden_GRU_2)

# hidden_2 = Dense(20)(input_A)
# hidden_flatten = Flatten()(input_A)

# concat = concatenate([hidden_1,hidden_flatten])
# output = Dense(3,activation="sigmoid")(concat)

# model = Model(inputs=[input_A,input_B],outputs=[output])


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),loss=tf.keras.losses.SparseCategoricalCrossentropy,metrics='val_loss')


In [ ]:
model.summary()

In [ ]:
trasad = tf.Variable(trasad)
travas = tf.Variable(travas)

In [ ]:
model.fit(trasad,tf.constant(y_train[:5]),epochs=1,validation_data=(travas,tf.constant(y_vaild[:5])))

ValueError: Failed to find data adapter that can handle input: <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>, <class 'tensorflow.python.framework.ops.EagerTensor'>

In [ ]:
tf.constant(y_train)

<tf.Tensor: shape=(29412, 3), dtype=uint8, numpy=
array([[0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]], dtype=uint8)>

In [ ]:
tf.data.Dataset.from_tensor_slices(train_df)

# tf.keras.utils.get_file("./input/train.csv",origin=".")

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).